In [52]:
import psycopg2
# Connect to your postgres DB
passward = input("Enter your password: ")

conn = psycopg2.connect(
dbname="final_project",
user="postgres",
password=passward,
host="localhost",
port="5432"
)
cur = conn.cursor()

Enter your password:  0128


In [3]:
# Create a new table
cur.execute(
    "CREATE TABLE population_growth ("
    "id SERIAL PRIMARY KEY, "
    "city varchar, "
    "country varchar, "
    "continent varchar, "
    "population_2024 integer, "
    "population_2023 integer, "
    "growth_rate decimal)"
)
# Commit the changes
conn.commit()

In [53]:
import csv

with open('/Users/hansaesbyeol/Desktop/final_p/Project_DataAnalysis/Wprld population growth rate by cities 2024.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        values = [str(d) for d in row[:6]]
        values[0] = "'" + values[0] + "'"
        values[1] = "'" + values[1] + "'"
        values[2] = "'" + values[2] + "'"
        
        values = ", ".join(values)

        cur.execute(
            "INSERT INTO population_growth "
            "(city, country, continent, population_2024, population_2023, growth_rate) "
            "VALUES ("+values+")"
        )
    conn.commit()


In [7]:
# Create a new table
cur.execute(
    "CREATE TABLE global_country_development ("
    "id SERIAL PRIMARY KEY, "
    "country varchar, "
    "average_score decimal, "
    "safety_security decimal, "
    "personel_freedom decimal, "
    "governance decimal, "
    "social_capital decimal, "
    "investment_enviroment decimal, "
    "enterprise_conditions decimal, "
    "market_access_infrastructure decimal, "
    "economic_quality decimal, "
    "living_conditions decimal, "
    "health decimal, "
    "education decimal, "
    "national_enviroment decimal)"
)
# Commit the changes
conn.commit()

In [36]:
import csv

with open('/Users/hansaesbyeol/Desktop/final_p/Project_DataAnalysis/2023 Global Country Development & Prosperity Index.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        values = [str(d) for d in row[:14]]
        values[0] = "'" + values[0] + "'"
        values = ", ".join(values)
        
        cur.execute(
            "INSERT INTO global_country_development "
            "(country, average_score, safety_security, personel_freedom, governance, social_capital, investment_enviroment, enterprise_conditions, "
            "market_access_infrastructure, economic_quality,living_conditions, health, education, national_enviroment) "
            "VALUES ("+values+")"
        )
        
    conn.commit()